<a href="https://colab.research.google.com/github/xarvel/DataScience/blob/master/birds_tfrec_convert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O dataset.zip https://www.dropbox.com/s/xkfeugecrkqjpks/100-bird-species.zip?dl=0
!unzip -q dataset.zip -d dataset

--2022-11-22 16:53:48--  https://www.dropbox.com/s/xkfeugecrkqjpks/100-bird-species.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xkfeugecrkqjpks/100-bird-species.zip [following]
--2022-11-22 16:53:49--  https://www.dropbox.com/s/raw/xkfeugecrkqjpks/100-bird-species.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc541eb8edb3ef0b8f328915b79c.dl.dropboxusercontent.com/cd/0/inline/BxONGeXOMqKPlrTkD1HTtxuNBKNg63gZ4nMqxKysWbwqic7JYc304ZA_O5UU5jhNdmdcFw8xnEG9_Saxix3c4il8feX34ttixyYqN1GvthIKuysTjT_MWlBo45DfzgevhSIABML9-gJ28uKmJiBVrAOmaijiYqAtzl1JFUnBkAItWA/file# [following]
--2022-11-22 16:53:49--  https://uc541eb8edb3ef0b8f328915b79c.dl.dropboxusercontent.com/cd/0/inline/BxONGeXOMqKPlrTkD1HTtxuNBKNg63gZ4nMqxKysWbwqic7J

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

import pathlib
import numpy as np

import os

dataset_path = 'dataset';

text_labels = sorted([name for name in os.listdir(dataset_path)])

train_data_root = pathlib.Path(dataset_path)
train_images_paths = [str(path) for path in list(train_data_root.glob('*/*.jpg'))]
train_images_text_labels = [pathlib.Path(path).parent.name for path in train_images_paths]
train_images_numerical_labels = [text_labels.index(text_label) for text_label in train_images_text_labels]
train_images_onehot_labels = [to_categorical(text_labels.index(text_label), len(text_labels), dtype='int64') for text_label in train_images_text_labels]

In [3]:
images_raw_ds =  tf.data.Dataset.from_tensor_slices(train_images_paths).map(tf.io.read_file)
images_ds = images_raw_ds.map(lambda image: tf.image.decode_jpeg(image, channels=3)).prefetch(1000)

text_labels_ds = tf.data.Dataset.from_tensor_slices(train_images_text_labels)
numerical_labels_ds = tf.data.Dataset.from_tensor_slices(train_images_numerical_labels)
onehot_labels_ds = tf.data.Dataset.from_tensor_slices(train_images_onehot_labels)
dataset = tf.data.Dataset.zip((images_ds, text_labels_ds, onehot_labels_ds, numerical_labels_ds)) 

In [4]:
def _bytes_feature(value):
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy()
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def serialize_train_example(image, label_text, label_onehot, label_number):
  image_raw = tf.image.encode_jpeg(image)
  
  feature = {
    'image': _bytes_feature(image_raw),
    'height': _int64_feature(image.shape[0]),
    'width': _int64_feature(image.shape[1]),
    'channels': _int64_feature(image.shape[2]),
    'label_text': _bytes_feature(label_text),
    'label_onehot': tf.train.Feature(int64_list=tf.train.Int64List(value=label_onehot)),
    'label_number': _int64_feature(label_number),
  }

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

def tf_serialize_test_example(image, label_text, label_one_hot, label_number):
  tf_string = tf.py_function(
    serialize_train_example,
    (image, label_text, label_one_hot, label_number), 
    tf.string)     
  return tf.reshape(tf_string, ()) 

def save_tf_record(dataset, filename):
  serialized_dataset = dataset.map(tf_serialize_test_example)
  tfrec = tf.data.experimental.TFRecordWriter(filename)
  tfrec.write(serialized_dataset)
  
save_tf_record(dataset, 'birds-dataset.tfrec')   

Instructions for updating:
To write TFRecords to disk, use `tf.io.TFRecordWriter`. To save and load the contents of a dataset, use `tf.data.experimental.save` and `tf.data.experimental.load`


In [5]:
def split_tfrecord(tfrecord_path, split_size):
    with tf.Graph().as_default(), tf.Session() as sess:
        ds = tf.data.TFRecordDataset(tfrecord_path).batch(split_size)
        batch = ds.make_one_shot_iterator().get_next()
        part_num = 0
        while True:
            try:
                records = sess.run(batch)
                part_path = tfrecord_path + '.{:03d}'.format(part_num)
                with tf.python_io.TFRecordWriter(part_path) as writer:
                    for record in records:
                        writer.write(record)
                part_num += 1
            except tf.errors.OutOfRangeError: break